In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# importo los .py
import sys
sys.path.append("auxiliares")
import config
import feature_eng

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [ ]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [ ]:
#Manual
prefijo_arch_entrada = "datasets/emp3_sellout_base_cluster_manual_"
prefijo_arch_salida = "datasets/emp3_sellout_fe_cluster_manual_"
num_clusters=94

#Time Warping
#prefijo_arch_entrada = "datasets/emp3_sellout_base_cluster_warp_"
#prefijo_arch_salida = "datasets/emp3_sellout_fe_cluster_warp_"

arch_productos = "datasets/emp3_sellout_producto_base.csv"
arch_clientes = "datasets/emp3_sellout_cliente_base.csv"
arch_cat1 = "datasets/emp3_sellout_cat1_base.csv"
arch_cat2 = "datasets/emp3_sellout_cat2_base.csv"
arch_cat3 = "datasets/emp3_sellout_cat3_base.csv"

In [ ]:
df_productos = pd.read_csv(arch_productos)
df_productos = df_productos.rename(columns={"product_id_tn":"tn"})
df_productos = df_productos[["product_id","periodo","tn"]] # limito solamente a columnas de interes
df_productos.info()

In [ ]:
df_clientes = pd.read_csv(arch_clientes)
df_clientes.info()

In [ ]:
df_cat1 = pd.read_csv(arch_cat1)
df_cat1.info()

In [ ]:
df_cat2 = pd.read_csv(arch_cat2)
df_cat2.info()

In [ ]:
df_cat3 = pd.read_csv(arch_cat3)
df_cat3.info()

In [ ]:
for i in range(0,num_clusters):
    print("Cluster", i)
    
    df_sellout = pd.read_csv(prefijo_arch_entrada + str(i) + ".csv")
    
    #Variable Target
    df_sellout = feature_eng.crear_target_prod_cust(df_sellout)
    
    #Nuevos Features
    df_sellout = feature_eng.crear_nuevos_features(df_sellout)
    
    #Features temporales
    df_sellout = feature_eng.crear_features_temporales("prod_cust",df_sellout, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_productos = feature_eng.crear_features_temporales("product_id",df_productos,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_clientes = feature_eng.crear_features_temporales("customer_id",df_clientes,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat1 = feature_eng.crear_features_temporales("cat1",df_cat1,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat2 = feature_eng.crear_features_temporales("cat2",df_cat2, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat3 = feature_eng.crear_features_temporales("cat3",df_cat3, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)

    #Joins
    print(len(df_sellout))
    df_productos_para_merge = df_productos.rename(columns={"tn":"product_id_tn"}) #revierto cambio para evitar colisiones
    
    df_sellout = df_sellout.merge(df_productos_para_merge,how="left",on=["product_id","periodo"])
    df_sellout = df_sellout.merge(df_clientes,how="left",on=["customer_id","periodo"])
    df_sellout = df_sellout.merge(df_cat1,how="left",on=["cat1","periodo"])
    df_sellout = df_sellout.merge(df_cat2,how="left",on=["cat2","periodo"])
    df_sellout = df_sellout.merge(df_cat3,how="left",on=["cat3","periodo"])
    print(len(df_sellout))
    
    df_sellout.to_csv(prefijo_arch_salida + str(i) + ".csv", index=False)

In [ ]:
df_sellout

## Verificaciones

In [ ]:
df_sellout.info()

In [ ]:
df_sellout.isna().sum()

In [ ]:
df_sellout.columns

In [ ]:
len(df_sellout.columns)